In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# recfldgrn List

In [2]:
# def traverse(o, tree_types=(list, tuple), index = None, nest_layer = 100):
#     if isinstance(o, tree_types) and nest_layer > 0:
#         for idx, value in enumerate(o):
#             new_index = index + [idx] if type(index) == list else [idx]
#             for subvalue in traverse(value, tree_types, new_index, nest_layer - 1):
#                 yield subvalue
#     else:
#         if not isinstance(o, tree_types): 
#             length = None
#         else:
#             length = len(o)
#         yield index, length, o
        
# from recfldgrn.utils import traverse
from fieldnn.utils.layerfn import traverse

In [3]:
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder

import os

In [4]:
TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)
recfldgrn_list = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
recfldgrn_list

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn']

# One recfldgrn

In [5]:
# recfldgrn = 'PNSectSent@Sentence-Tk@TknzLLMGrn'; prefix_ids = ['P', 'EC', 'PN', 'PNSect']
# recfldgrn = 'A1C@DT-DTDftGrn'; prefix_ids = ['P', 'EC']
# full_recfldgrn = '-'.join(prefix_ids) + '-' + recfldgrn
# print(full_recfldgrn)

## Load df_Pat

In [6]:
##############
# recfldgrn = 'PNSectSent@Sentence-Tk@TknzLLMGrn'
recfldgrn = 'A1C@DT-DTDftGrn'
##############

In [7]:
tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)
df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
full_recfldgrn = df_Pat.columns[0]
suffix = full_recfldgrn.split('_')[-1]
assert recfldgrn in full_recfldgrn
print(full_recfldgrn)
print(suffix)
df_Pat

P-EC-A1C@DT-DTDftGrn_idx
idx


,P-EC-A1C@DT-DTDftGrn_idx
PID,
P0,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P1,"[[[16, 27, 38, 70, 80, 95, 138]], [[2, 23, 36,..."
P2,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P3,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P4,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P5,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P6,"[[[16, 27, 48, 70, 76, 97, 138]], [[2, 23, 36,..."
P7,"[[[16, 27, 42, 70, 77, 96, 138]]]"


In [8]:
print(full_recfldgrn)

P-EC-A1C@DT-DTDftGrn_idx


## get Batch

In [9]:
###################### take this as given
batch_PID_order = ['P0', 'P2', 'P1', 'P3', 'P4', 'P5', 'P6', 'P7']
######################

In [10]:
df_batch = df_Pat.loc[batch_PID_order]
df_batch

,P-EC-A1C@DT-DTDftGrn_idx
PID,
P0,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P2,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P1,"[[[16, 27, 38, 70, 80, 95, 138]], [[2, 23, 36,..."
P3,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P4,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P5,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P6,"[[[16, 27, 48, 70, 76, 97, 138]], [[2, 23, 36,..."
P7,"[[[16, 27, 42, 70, 77, 96, 138]]]"


In [11]:
values_list = df_batch[full_recfldgrn].to_list()
# values_list

In [12]:
from recfldgrn.utils import convert_relational_list_to_numpy
# from fieldnn.utils.layerfn import convert_relational_list_to_numpy

In [13]:
# import numpy as np

# def convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix):
#     o = values_list
#     layer_num = len(new_full_recfldgrn.split('-'))
#     layers = new_full_recfldgrn.replace(suffix, '').split('-')
#     # L = [len(values_list)] 

#     D = {}

#     # (1) from first layer: 0
#     idx = 0
#     layer_parents = layers[:idx + 1]
#     layer_children = layers[idx + 1]
#     len_name = f'{"2".join(layer_parents)}_ln{layer_children}'
#     len_np = np.array(len(values_list))
#     len_shapes = [len_np.max()] # from layer 0, prepare for layer 1. 
#     D[len_name] = len_np

#     # (2) from 1 - last one layers
#     for idx in range(1, layer_num - 1):
#         output = list(traverse(o, nest_layer = idx))
#         # print(output)
#         # data = np.zeros(L)
#         # print('\n\n')
#         # print(idx)
#         # print(output)

#         layer_parents = layers[:idx + 1]
#         layer_children = layers[idx + 1]
#         len_name = f'{"-".join(layer_parents)}_ln{layer_children}'
#         # print(len_name)

#         locidx  = [i[0] for i in output]
#         length = [i[1] for i in output]
#         # values = [i[2] for i in output]

#         len_np = np.zeros(len_shapes).astype(int)
#         # print(len_np.shape, '<---- len_np.shape')
#         for locidx, length, _ in output:
#             len_np[tuple(locidx)] = int(length)
#         # print(len_np)
#         len_shapes.append(len_np.max())
#         # print(len_shapes, '<---- next len_np.shape')
#         # print(length)
#         # print()
#         D[len_name] = len_np

#     # (3) for the data
#     idx = layer_num
#     name = new_full_recfldgrn
#     data = np.zeros(len_shapes) # don't convert it to int for now. 
#     output = list(traverse(o, nest_layer = idx))
#     for locidx, _, value in output:
#         data[tuple(locidx)] = value
#         # print(locidx, value)
#         # print(data[tuple(locidx)])# = value
#     data.shape
#     D[name] = data.astype(int)
    
#     return D


In [14]:
print(full_recfldgrn)

P-EC-A1C@DT-DTDftGrn_idx


In [15]:
df_batch

,P-EC-A1C@DT-DTDftGrn_idx
PID,
P0,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P2,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P1,"[[[16, 27, 38, 70, 80, 95, 138]], [[2, 23, 36,..."
P3,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P4,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P5,"[[[2, 23, 36, 68, 73, 81, 137]], [[2, 23, 36, ..."
P6,"[[[16, 27, 48, 70, 76, 97, 138]], [[2, 23, 36,..."
P7,"[[[16, 27, 42, 70, 77, 96, 138]]]"


In [16]:
new_full_recfldgrn = 'B-' + full_recfldgrn
values_list = df_batch[full_recfldgrn].to_list()
suffix = full_recfldgrn.split('_')[-1]
print(suffix)
print(new_full_recfldgrn)
values_list

idx
B-P-EC-A1C@DT-DTDftGrn_idx


[[[[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[16, 26, 63, 69, 74, 95, 138]],
  [[16, 27, 42, 70, 77, 96, 138]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[16, 27, 44, 70, 79, 96, 138]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[16, 27, 48, 70, 76, 97, 138]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23, 36, 68, 73, 81, 137]],
  [[2, 23,

In [17]:
D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)

for k, v in D.items(): print(k, v.shape)

B_lnP ()
B-P_lnEC (8,)
B-P-EC_lnA1C@DT (8, 87)
B-P-EC-A1C@DT_lnDTDftGrn_ (8, 87, 1)
B-P-EC-A1C@DT-DTDftGrn_idx (8, 87, 1, 7)


In [18]:
tensor_idx = D[new_full_recfldgrn]# .shape
print(new_full_recfldgrn)
print(tensor_idx.shape)

B-P-EC-A1C@DT-DTDftGrn_idx
(8, 87, 1, 7)


In [22]:
new_full_recfldgrn

'B-P-EC-A1C@DT-DTDftGrn_idx'

In [21]:
tensor_idx.shape

(8, 87, 1, 7)

## Validation

In [19]:
tensor_idx[0][0]#[0][0][0]

array([[  2,  23,  36,  68,  73,  81, 137]])

In [20]:
values_list = df_batch[full_recfldgrn].to_list()
values_list[0][0]#[0][0][0]

[[1, 22, 35, 67, 72, 80, 136]]

In [21]:
tensor_idx[0][3]#[0][0][0]

array([[ 15,  25,  62,  68,  73,  94, 137]])

In [22]:
values_list = df_batch[full_recfldgrn].to_list()
values_list[0][3]#[0][0][0]

[[15, 25, 62, 68, 73, 94, 137]]

# Load Tensor 

In [23]:
###################### take this as given
batch_PID_order = ['P0', 'P2', 'P1', 'P3', 'P4', 'P5', 'P6', 'P7']
######################

In [24]:
TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)
recfldgrn_list = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
recfldgrn_list

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn']

In [28]:
recfldgrn_list = [
                  'P@age-AgeNumeDftGrn',
                  'P@basicInfo-basicInfoDftGrn',
    
                  'EC@BasicInfo-BasicDftGrn',
                  'EC@DT_min-DTDftGrn',
    
                  'A1C@DT-DTDftGrn',
                  'A1C@V-A1CNumeDftGrn',
                  
                  'Diag@DT-DTDftGrn',
                  'Diag@Value-DiagDftGrn',
                  
                  'PNSectSent@Sentence-Tk@TknzLLMGrn']

In [29]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx

In [30]:
for k, v in batch_rfg.items(): print(k, v.shape)

B-P@age-AgeNumeDftGrn_wgt (8, 19)
B-P@basicInfo-basicInfoDftGrn_idx (8, 2)
B-P-EC@BasicInfo-BasicDftGrn_idx (8, 87, 2)
B-P-EC@DT_min-DTDftGrn_idx (8, 87, 7)
B-P-EC-A1C@DT-DTDftGrn_idx (8, 87, 1, 7)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (8, 87, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (8, 87, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (8, 87, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (8, 87, 1, 15, 121, 11)
